In [1]:
import math
import torch
import torch.nn as nn
import torch.optim as optim

/home/dingkang/envs/nlp_a4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Refer to https://www.datacamp.com/tutorial/building-a-transformer-with-py-torch

In [2]:
class PositionWiseFFN(nn.Module):
    """Same MLP applied to all token(position) representations"""
    def __init__(self, emb_dim, ffn_dim):
        super().__init__()
        self.fc1 = nn.Linear(emb_dim, ffn_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(ffn_dim, emb_dim)
    
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [3]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_dim, max_seq_len):
        super().__init__()

        pe = torch.zeros(max_seq_len, emb_dim)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, emb_dim, 2).float() * -(math.log(10000.0) / emb_dim))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))
    
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


In [4]:
class MultiHeadAttention(nn.Module):
    def __init__(self, emb_dim, num_heads):
        super().__init__()
        assert emb_dim % num_heads == 0, "Embedding dimension must be divided by number of heads"

        # Dimensions initialization
        self.emb_dim = emb_dim
        self.num_heads = num_heads
        # all features are divided into multi head, each head have a part of features
        self.head_emb_dim = self.emb_dim // self.num_heads

        # Transformation matrixs
        self.W_q = nn.Linear(emb_dim, emb_dim)
        self.W_k = nn.Linear(emb_dim, emb_dim)
        self.W_v = nn.Linear(emb_dim, emb_dim)
        self.W_o = nn.Linear(emb_dim, emb_dim)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_emb_dim)

        # Mask scores (where positions are 0) with near negative inf
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        # Apply sofxmax to attention scores
        attn_scores = torch.softmax(attn_scores, dim=-1)

        # Get the final output
        output = torch.matmul(attn_scores, V)
        return output
    
    def split(self, x):
        # Reshape the input emb_dim (to multi-head, each head owns a part of input features) for multi-head attention
        batch_size, seq_len, emb_dim = x.size()
        # transpose to fix batch_size and num_heads, let seq_len, head_emb_dim participate in matrix multiplication
        return x.view(batch_size, seq_len, self.num_heads, self.head_emb_dim).transpose(1, 2)

    def combine(self, x):
        batch_size, num_heads, seq_len, head_emb_dim = x.size()
        # contiguous() ensures the memory layout of the tensor is contiguous
        return x.transpose(1, 2).contiguous().view(batch_size, seq_len, self.emb_dim)
    
    def forward(self, Q, K, V, mask=None):
        # Split input to multi heads
        Q = self.split(self.W_q(Q))
        K = self.split(self.W_k(K))
        V = self.split(self.W_v(V))

        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)

        # Combine outputs and apply transformation
        output = self.W_o(self.combine(attn_output))
        return output


In [5]:
class EncoderLayer(nn.Module):
    def __init__(self, emb_dim, num_heads, ffn_dim, dropout):
        super().__init__()
        self.self_atten = MultiHeadAttention(emb_dim, num_heads)
        self.ffn = PositionWiseFFN(emb_dim, ffn_dim)
        self.norm1 = nn.LayerNorm(emb_dim)
        self.norm2 = nn.LayerNorm(emb_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_atten(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ffn_output = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_output))
        return x

In [6]:
class DecoderLayer(nn.Module):
    def __init__(self, emb_dim, num_heads, ffn_dim, dropout):
        super().__init__()
        self.self_attn = MultiHeadAttention(emb_dim, num_heads)
        self.cross_attn = MultiHeadAttention(emb_dim, num_heads)
        self.ffn = PositionWiseFFN(emb_dim, ffn_dim)
        self.norm1 = nn.LayerNorm(emb_dim)
        self.norm2 = nn.LayerNorm(emb_dim)
        self.norm3 = nn.LayerNorm(emb_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        # Masked Multiple Head Attention
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ffn_output = self.ffn(x)
        x = self.norm3(x + self.dropout(ffn_output))
        return x

In [7]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, emb_dim, num_heads,
                num_layers, ffn_dim, max_seq_len, dropout):
        super().__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, emb_dim)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, emb_dim)
        self.positional_encoding = PositionalEncoding(emb_dim, max_seq_len)

        self.encoder_layers = nn.ModuleList([EncoderLayer(emb_dim, num_heads, ffn_dim, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(emb_dim, num_heads, ffn_dim, dropout) for _ in range(num_layers)])

        # No need for a softmax, because it is already included in the CrossEntropyLoss
        self.fc = nn.Linear(emb_dim, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
        
    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2) # 0s stand for padding tokens
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3) # unsqueeze(1) add new dimension at axis 1
        seq_len = tgt.size(1)
        nopeak_mask = (1-torch.triu(torch.ones(1, seq_len, seq_len), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        print(src_mask.shape)
        # inputEmb + PosEmb is implemented in self.positional_encoding())
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)
        
        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)
        
        output = self.fc(dec_output)
        return output

In [8]:
src_vocab_size = 50
tgt_vocab_size = 50
emb_dim = 18
num_heads = 3
num_layers = 2
ffn_dim = 24
max_seq_len = 6
dropout = 0.1

In [9]:
transformer = Transformer(src_vocab_size, tgt_vocab_size, emb_dim, num_heads, num_layers, ffn_dim, max_seq_len, dropout)

src_data = torch.randint(1, src_vocab_size, (8, max_seq_len))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (8, max_seq_len))  # (batch_size, seq_length)
src_data[:, 2:5] = 0

In [10]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(5):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    a = output.contiguous().view(-1, tgt_vocab_size)
    b = tgt_data[:, 1:].contiguous().view(-1)
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

torch.Size([8, 1, 1, 6])
Epoch: 1, Loss: 4.1492767333984375
torch.Size([8, 1, 1, 6])
Epoch: 2, Loss: 4.147047996520996
torch.Size([8, 1, 1, 6])
Epoch: 3, Loss: 4.219018459320068
torch.Size([8, 1, 1, 6])
Epoch: 4, Loss: 4.155802249908447
torch.Size([8, 1, 1, 6])
Epoch: 5, Loss: 4.134562015533447


In [11]:
transformer.eval()

# Generate random sample validation data
val_src_data = torch.randint(1, src_vocab_size, (8, max_seq_len))  # (batch_size, seq_length)
val_tgt_data = torch.randint(1, tgt_vocab_size, (8, max_seq_len))  # (batch_size, seq_length)

with torch.no_grad():

    val_output = transformer(val_src_data, val_tgt_data[:, :-1])
    val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), val_tgt_data[:, 1:].contiguous().view(-1))
    print(f"Validation Loss: {val_loss.item()}")

torch.Size([8, 1, 1, 6])
Validation Loss: 4.326568603515625


In [12]:
class BertEncoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, num_heads,
                num_layers, ffn_dim, max_seq_len, dropout):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, emb_dim)
        self.segment_embedding = nn.Embedding(2, emb_dim)
        self.encoder_layers = nn.ModuleList([EncoderLayer(emb_dim, num_heads, ffn_dim, dropout) for _ in range(num_layers)])
        self.pos_embedding = nn.Parameter(torch.randn(1, max_seq_len, emb_dim))

    def forward(self, tokens, segments):
        token_mask = (tokens != 0).unsqueeze(1).unsqueeze(2)
        x = self.token_embedding(tokens) + self.segment_embedding(segments)
        x = x + self.pos_embedding[:, :x.shape[1], :]
        for enc_layer in self.encoder_layers:
            x = enc_layer(x, token_mask)
        return x

In [13]:
vocab_size, emb_dim, ffn_dim, num_heads = 50, 18, 24, 3
num_layers, max_seq_len, dropout = 2, 10, 0.2
encoder = BertEncoder(vocab_size, emb_dim, num_heads, num_layers, ffn_dim, max_seq_len, dropout)

In [14]:
tokens = torch.randint(0, vocab_size, (2, 8))
segments = torch.tensor([[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]])
encoded_x = encoder(tokens, segments)
encoded_x.shape

torch.Size([2, 8, 18])

In [15]:
class MaskLM(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super().__init__()
        self.mlp = nn.Sequential(nn.Linear(emb_dim, emb_dim),
                                 nn.ReLU(),
                                 nn.LayerNorm(emb_dim),
                                 nn.Linear(emb_dim, vocab_size))
    
    def forward(self, x, pred_positions):
        num_pred_positions = pred_positions.shape[1]
        pred_positions = pred_positions.reshape(-1)
        batch_size = x.shape[0]
        batch_idx = torch.arange(0, batch_size)
        batch_idx = torch.repeat_interleave(batch_idx, num_pred_positions)
        # get only masked tokens' representation
        masked_x = x[batch_idx, pred_positions]
        masked_x = masked_x.reshape((batch_size, num_pred_positions, -1))
        mlm_Y_hat = self.mlp(masked_x)
        return mlm_Y_hat

In [16]:
mlm = MaskLM(vocab_size, emb_dim)
mlm_positions = torch.tensor([[1, 5, 2], [6, 1, 5]])
mlm_Y_hat = mlm(encoded_x, mlm_positions)
mlm_Y_hat.shape

torch.Size([2, 3, 50])

In [17]:
mlm_Y = torch.tensor([[7, 8, 9], [10, 20, 30]])
loss = nn.CrossEntropyLoss(reduction='none')
mlm_l = loss(mlm_Y_hat.reshape((-1, vocab_size)), mlm_Y.reshape(-1))
mlm_l.shape

torch.Size([6])

In [18]:
class NextSentencePred(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.output = nn.LazyLinear(2)
    
    def forward(self, x):
        return self.output(x)

In [19]:
encoded_x_new = torch.flatten(encoded_x, start_dim=1)
nsp = NextSentencePred(emb_dim)
nsp_y_hat = nsp(encoded_x_new)
nsp_y_hat.shape
# encoded_x_new.shape

/home/dingkang/envs/nlp_a4/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 2])

In [20]:
class BertModel(nn.Module):
    def __init__(self, vocab_size, emb_dim, num_heads,
                 num_layers, ffn_dim, max_seq_len, dropout):
        super().__init__()
        self.encoder = BertEncoder(vocab_size, emb_dim, num_heads, num_layers, ffn_dim, max_seq_len, dropout)
        self.hidden = nn.Sequential(nn.LazyLinear(emb_dim),
                                    nn.Tanh())
        self.mlm = MaskLM(vocab_size, emb_dim)
        self.nsp = NextSentencePred()

    def forward(self, tokens, segments, pred_positions=None):
        encoded_x = self.encoder(tokens, segments)
        if pred_positions is not None:
            mlm_y = self.mlm(encoded_x, pred_positions)
        else:
            mlm_y = None
        # 0 is the index of the '<cls>' token
        nsp_y = self.nsp(self.hidden(encoded_x[:, 0, :]))
        return encoded_x, mlm_y, nsp_y

In [21]:
import pandas as pd

types_dict = {'PK':0, 'MR': 1, 'KL':2, 'NV':3, 'WA':4, 'LG':5, 'HO':6, 'GR':7, 'REC':8, 'PGK':9}
df = pd.read_csv("archaeology.csv")
df['type'] = df['Aardspoor'].map(types_dict)
data = df['WKT'][:1000]